# Design Requirements:


The pintle injector needs to meet the following requirements:

1. Oxidizer: Liquid Oxygen (LOX)
2. Fuel: 70% Isopropyl Alcohol, 30% water (IPA)
3. Meet mass flow rates at a pressure differential between 15-25 % of chamber pressure: 
    - Engine chamber pressure 350 psi (as designed)
    - Target pressure differential: 53-88 psi   
4. LOX mass flow rate of 0.9 lbm/s (0.4082 kg/s)
5. Fuel mass flow rate of 1.16 lbm/s (0.5262 kg/s)

Additional physical size and machining constraints

1. Max outer diameter for Annulus, 0.55 in (13.97 mm)
2. Min Pintle wall thickness 0.08 in (2.032 mm)
    - A larger wall thickness is preferred, target .12 in (3.048 mm)
    
    

# Design considerations

1. The flow paths need to be increased in size for both the Pintle and Annulus. Both underperformed and will need larger orifices. 

2. The test jig used for the Pintle V1 needs to be redesigned. The V1 design only contains 1 flow path to the fuel chamber behind the annulus, while the actual engine contains 3. The current dimensions of the singular flow path (0.126"$\emptyset$) are such that it introduces sizable pressure losses which will not be present in the actual engine.
 - The diameter of the flow path into the fuel chamber needs to be increased to 0.25"$\emptyset$ which more closely resembles the actual geometry.
 
 




# Design Changes for Version 2

## Dimension Changes
The primary changes to the pintle and annulus test jig dealt with the size of the pintle and the related orifices. The exact values used in both designs can be seen in Table 1.

<img src="./Images/Pintle_dims_v1-2.PNG" align="center" width="500"/>

>Table 1: Comparison of critical design dimensions for pintle and annulus flow areas.




## Methods

### Pintle Design Tool input requirements
The new injector dimensions were arrived at using the Version 2 section of the Pintle Design Tool. See Figure 1 dimension locations. 

The tool uses the following inputs:
- fluids:
    - Propellant mass flow rates
    - Propellant fluid properties
- Pintle Geometry
   - **Pintle OD**
   - **Pintle Exit Diameters**
   - **Pintle ID**
   
- Annulus Geometry
    - **Annulus OD**
    - Annulus Entrance Diameter
    - Chamber Diameter

>_Note: Dimensions in **bold** were assumed to be critical and were the subject of most of the changes to geometry when exploring possible designs_



<img src="./Images/pintle_dimensions(1).PNG" align="center" width="500"/>

> Figure 1: cross section view of genaric pintle V1 & V2 geometry.


### Getting Started
When approaching the redesign of the pintle, it was noted that the thickness of the inner diameter combined with the total wall thickness of the pintle comprise the Pintle OD. This dimension sets the lower limit on size for the annulus.  

The logical sequence for determining the design is to first determine a feasible geometry for the pintle (LOX flow path), and then determine the needed Annulus OD orifice size.


### Determining the Pintle dimensions:

A standard drill size chart was used to determine the range of easily manufactured sizes for both the inner diameter for the pintle and the exit orifices.

Starting several drill sizes larger than those used for the V1 pintle, possible inner and exit diameters were entered into the design tool. 

The design tool used the given inputs along with estimated loss coefficients to determine the approximate pressure drop. For more information on the equations used in these calculations see [V1-01 Pintle Flow Calculations and Analysis](https://github.com/psas/liquid-engine-test-stand/blob/master/Pintle/Pintle%20Injector%20Design%20Documentation/V1-01%20Pintle%20Flow%20Calculations%20and%20Analysis.ipynb) which can be found in the Pintle injector design documentation folder of the Liquid-Engine-Test-Stand repo on PSAS's Github page.

### Determining the Annulus dimension

Having determined the potential Pintle ID, the Pintle OD was calculated by from $2t_w * P_{ID}$ where $t_w$ is the pintle wall thickness and $P_{ID}$ is the pintle ID.

Given the relatively small size of the Pintle, standard drill sizes were used as inputs for the Annulus OD. The first drill size checked was one size larger than the Pintle OD. consecutively larger drill sizes were tried until the calculated pressure loss dropped into the desired range.






# Validation of the design

The geometry found using the Pintle Design Tool was used to create a CFD model using Star CCM+.

The CFD model used the same mass flow requirements and geometry as was used to determine the V2 pintle geometry. The pressure loss found in the CFD analysis were found to be within 2 psi of the Pintle Design Tool.

The one potential issue noted with the design during the CFD analysis was uneven exit velocity (Fig 2). 

<img src="./Images/Velocity_profile_1.png" align="center" width="500"/>

> Figure 2: Uneven exit velocity in annular region. The section of lowered velocity is directly above the inlet to the annular chamber.

Minor changes to the geometry were experimented with to create a more even flow pattern. It was solved in the model$^\dagger$ by increasing the distance between the inlet and the top of the annular chamber.



__________________________
$^\dagger$_This did not actually solve the issue and uneven flow was observed while hydro testing the annulus. See the V2 postmortem and V3 design documentation for details_





# V2 Pintle Calculator


In [1]:
import math
from mpmath import *
import numpy as np
from matplotlib import pyplot as plt


# Desired flow parameters
pintle_mdot = 0.4082    # kg/s  , desired Pintle mass flow rate
annulus_mdot = 0.5262   # kg/s  , desired Annulus mass flow rate

# Fluid properties

fluid_pintle  = 'water'
fluid_annulus = 'water'

if fluid_pintle == 'water':
    pintle_rho = 999.7          # kg/m^3
    pintle_dynvisc = 0.001307   # N*s/m^2

elif fluid_pintle == 'LOX':
    pintle_rho     = 1141        # kg/m^3
    pintle_dynvisc = 0.0001971   # N*s/m^2

if fluid_annulus == 'water':
    annulus_rho       = 999.7          # kg/m^3
    annulus_dynvisc   = 0.001307       # N*s/m^2

elif fluid_annulus == 'IPA':
    annulus_rho       = 877.8         # kg/m^3  
    annulus_dynvisc   = 0.002         # N*s/m^2
    

# V_2.0 geometry

def inch_to_meters(inch):
    meter = inch / 39.36996
    return(meter)

pintle_entrance_dia = inch_to_meters(0.402)   
pintle_inner_dia = inch_to_meters(0.221)      
pintle_exit_dia = inch_to_meters(0.0595)      

# V_2.0 Annulus

pipe_diameter         = inch_to_meters(0.402)    # m (0.402 " pipe diameter)
annulus_entrance_dia  = inch_to_meters(0.25)     # m (0.25 in) 
annulus_chamber_dia   = inch_to_meters(0.865)   # m (0.865 in)
annulus_inner_dia     = inch_to_meters(0.421)   # m (0.281 in) The outer diameter of the pintle
annulus_outer_dia     = inch_to_meters(0.4688)   # m (0.415 in)
annulus_hydrolic_dia  = annulus_outer_dia - annulus_inner_dia

# Area calculations

pintle_entrance_area  = math.pi*(pintle_entrance_dia/2)**2        # m^2 , flow area 1
pintle_inner_area     = math.pi*(pintle_inner_dia/2)**2        # m^2 , flow area 2
pintle_exit_area      = 12 * math.pi*(pintle_exit_dia/2)**2   # m^2 , flow area 3, 12 small diameter holes

pipe_area             = math.pi*(pipe_diameter/2)**2        # m^2 , flow area 1
annulus_entrance_area = math.pi*(annulus_entrance_dia/2)**2        # m^2 , flow area 2
annulus_chamber_area  = math.pi*(annulus_chamber_dia/2)**2        # m^2 , flow area 3
annulus_inner_area    = math.pi*(annulus_inner_dia/2)**2   # m^2 , flow area 4, 12 small diameter holes
annulus_outer_area    = math.pi*(annulus_outer_dia/2)**2   # m^2 , flow area 4, 12 small diameter holes
annulus_exit_area     = annulus_outer_area - annulus_inner_area

# velocity calculations

pintle_velocity_entrance  = pintle_mdot / (pintle_rho * pintle_entrance_area)  
pintle_inner_velocity     = pintle_mdot / (pintle_rho * pintle_inner_area)  
pintle_velocity_exit      = pintle_mdot / (pintle_rho * pintle_exit_area) 

pipe_fuel_velocity        = annulus_mdot / (annulus_rho * pipe_area)
annulus_entrance_velocity = annulus_mdot / (annulus_rho * annulus_entrance_area) 
annulus_chamber_velocity  = annulus_mdot / (annulus_rho * annulus_chamber_area)  
annulus_exit_velocity     = annulus_mdot / (annulus_rho * annulus_exit_area) 

# Entrance Dynamic Pressure

pintle_dynamic_pressure_in = (pintle_rho*pintle_velocity_entrance**2/2)* 0.1450377/1000
annulus_dynamic_pressure_in = (annulus_rho*pipe_fuel_velocity**2/2)* 0.1450377/1000

# Reynolds numbers

pintle_re_1 = (pintle_rho*pintle_velocity_entrance*pintle_entrance_dia)/pintle_dynvisc 
pintle_re_2 = (pintle_rho*pintle_inner_velocity*pintle_inner_dia)/pintle_dynvisc 
pintle_re_3 = (pintle_rho*pintle_velocity_exit*pintle_exit_dia)/pintle_dynvisc 

annulus_re_1 = (annulus_rho*pipe_fuel_velocity*pipe_diameter)/annulus_dynvisc 
annulus_re_2 = (annulus_rho*annulus_entrance_velocity*annulus_entrance_dia)/annulus_dynvisc 
annulus_re_3 = (annulus_rho*annulus_chamber_velocity*annulus_chamber_dia)/annulus_dynvisc 
annulus_re_4 = (annulus_rho*annulus_exit_velocity*annulus_hydrolic_dia)/annulus_dynvisc 

# Loss coefficents; taken from Fluids textbook

pintle_k_1 = 0.07   # loss coefficient for first entrance region
pintle_k_2 = 2      # inner region  (originally using 2, 3 puts this inline with experiment)
pintle_k_3 = 1      # Exit region (originally used value of 1, 4 puts this inline with experiment)

annulus_k_1 = 0.38      # Pipe to test jig (0.38)
annulus_k_2 = 1         # Test jig to fuel chamber (1)
annulus_k_3 = 0.04      # Fuel Chamber to annulus (0.04)
annulus_k_4 = 1         # Exit region (1)

# calculated head losses in kPa

pintle_hL_1 = (pintle_k_1*0.5*pintle_rho*pintle_velocity_entrance**2)/1000 
pintle_hL_2 = (pintle_k_2*0.5*pintle_rho*pintle_inner_velocity**2)/1000 
pintle_hL_3 = (pintle_k_3*0.5*pintle_rho*pintle_velocity_exit**2)/1000 
pintle_hl_tot = pintle_hL_1 + pintle_hL_2+pintle_hL_3

annulus_hL_1 = (annulus_k_1*0.5*annulus_rho*pipe_fuel_velocity**2)/1000 
annulus_hL_2 = (annulus_k_2*0.5*annulus_rho*annulus_entrance_velocity**2)/1000 
annulus_hL_3 = (annulus_k_3*0.5*annulus_rho*annulus_chamber_velocity**2)/1000 
annulus_hL_4 = (annulus_k_4*0.5*annulus_rho*annulus_exit_velocity**2)/1000 
annulus_hl_tot = annulus_hL_1 + annulus_hL_2 + annulus_hL_3 + annulus_hL_4

#convert to psi from kPa

pintle_hL_1_psi = pintle_hL_1 * 0.1450377    
pintle_hL_2_psi = pintle_hL_2 * 0.1450377    
pintle_hL_3_psi = pintle_hL_3 * 0.1450377    
pintle_hL_total_psi = (pintle_hL_1_psi+pintle_hL_2_psi+pintle_hL_3_psi) #* 2.5 #correction factor  

pintle_K_L = (2*pintle_hl_tot*1000)/(pintle_rho*pintle_velocity_exit**2)

annulus_hL_1_psi = annulus_hL_1 * 0.1450377  
annulus_hL_2_psi = annulus_hL_2 * 0.1450377    
annulus_hL_3_psi = annulus_hL_3 * 0.1450377    
annulus_hL_4_psi = annulus_hL_4 * 0.1450377    
annulus_hL_total_psi = (annulus_hL_1_psi+annulus_hL_2_psi+annulus_hL_3_psi+annulus_hL_4_psi)  

annulus_K_L = (2*annulus_hl_tot*1000)/(annulus_rho*annulus_exit_velocity**2)

print("Calculated flow values\n")
print('Pintle calculations if using %s\n' %fluid_pintle)
print('Pintle entrance velocity    = %8.3f m/s' % pintle_velocity_entrance)
print('Pintle inner velocity       = %8.3f m/s' % pintle_inner_velocity)
print('Pintle exit velocity        = %8.3f m/s' % pintle_velocity_exit)

print('Pintle entrance Reynolds #  = %8.0f' % pintle_re_1)
#print('Pintle Reynolds # 2         = %10.0f' % pintle_re_2)
print('Pintle exit Reynolds #      = %8.0f' % pintle_re_3)
print('Pintle headloss 1           = %8.3f psi' % pintle_hL_1_psi)
print('Pintle headloss 2           = %8.3f psi' % pintle_hL_2_psi)
print('Pintle headloss 3           = %8.3f psi' % pintle_hL_3_psi)
print('Pintle Total head loss      = %8.2f psi\n' % pintle_hL_total_psi)
print('Pintle  Total head loss     = %8.3f kPa' % pintle_hl_tot)

print('Pintle  KL total            = %8.3f \n' % pintle_K_L)

print('Pintle Entrance Dynamic Pressure = %8.3f \n' % pintle_dynamic_pressure_in)



print('Annulus calculations if using %s\n' %fluid_annulus)
print('Pipe fuel velocity          = %8.3f m/s' % pipe_fuel_velocity)
print('annulus entrance velocity   = %8.3f m/s' % annulus_entrance_velocity)
print('annulus inner velocity      = %8.3f m/s' % annulus_chamber_velocity)
print('annulus exit velocity       = %8.3f m/s' % annulus_exit_velocity)

print('annulus entrance Reynolds # = %8.0f' % annulus_re_1)
#print('annulus Reynolds # 2        = %10.3f' % annulus_re_2)
print('annulus exit Reynolds #     = %8.0f' % annulus_re_3)

#print('annulus headloss 1          = %8.3f psi' % annulus_hL_1_psi)
#print('annulus headloss 2          = %8.3f psi' % annulus_hL_2_psi)
#print('annulus headloss 3          = %8.3f psi' % annulus_hL_3_psi)
#print('annulus headloss 3          = %8.3f psi' % annulus_hL_4_psi)
print('annulus Total head loss     = %8.3f kPa' % annulus_hl_tot)
#print('annulus Total head loss     = %8.2f psi\n' % annulus_hL_total_psi)

print('annulus KL total            = %8.3f \n' % annulus_K_L)

print('Annulus Entrance Dynamic Pressure = %8.3f \n' % annulus_dynamic_pressure_in)



Calculated flow values

Pintle calculations if using water

Pintle entrance velocity    =    4.986 m/s
Pintle inner velocity       =   16.499 m/s
Pintle exit velocity        =   18.968 m/s
Pintle entrance Reynolds #  =    38945
Pintle exit Reynolds #      =    21927
Pintle headloss 1           =    0.126 psi
Pintle headloss 2           =   39.470 psi
Pintle headloss 3           =   26.084 psi
Pintle Total head loss      =    65.68 psi

Pintle  Total head loss     =  452.851 kPa
Pintle  KL total            =    2.518 

Pintle Entrance Dynamic Pressure =    1.803 

Annulus calculations if using water

Pipe fuel velocity          =    6.428 m/s
annulus entrance velocity   =   16.620 m/s
annulus inner velocity      =    1.388 m/s
annulus exit velocity       =   24.423 m/s
annulus entrance Reynolds # =    50202
annulus exit Reynolds #     =    23331
annulus Total head loss     =  444.118 kPa
annulus KL total            =    1.490 

Annulus Entrance Dynamic Pressure =    2.995 

